In [ ]:
#!pip install PyCustomFocus

In [ ]:
#!pip install s2ism

In [ ]:
#!pip install torch

In [ ]:
# !pip install nanopyx
# !pip install cupy

In [ ]:
#!pip install brighteyes_ism

In [ ]:
#pip install opencv-python

In [ ]:
import numpy as np
import scipy
import itertools
import matplotlib.pyplot as plt
from tqdm import tqdm
import scipy.fftpack
#import csv 
#import os
import cv2
import tifffile
from PIL import Image
import h5py
import array 
from matplotlib.patches import Rectangle
from scipy.optimize import curve_fit
from skimage.feature import peak_local_max
import os
from os import path

import brighteyes_ism.analysis.APR_lib as apr
import brighteyes_ism.analysis.APR_lib as apr
import brighteyes_ism.analysis.FRC_lib as frc
import brighteyes_ism.analysis.Deconv_lib as dec
import brighteyes_ism.analysis.FocusISM_lib as fism
import brighteyes_ism.analysis.Graph_lib as gra
import brighteyes_ism.analysis.Tools_lib as tool
import brighteyes_ism.dataio.mcs as mcs
import brighteyes_ism.analysis.Deconv_lib as dec
import brighteyes_ism.simulation.PSF_sim as sim
from scipy.ndimage import fourier_shift, shift 

In [ ]:
# LIBRERIE PER LA SURF:

# NOTA:
# nanopyx ha bisogno della libreria cupy che è la copia di numpy ottimizzata per GPU nvidia

# from nanopyx import eSRRF
# from nanopyx import SRRF
# from nanopyx.core.transform.sr_temporal_correlations import calculate_eSRRF_temporal_correlations

In [ ]:
dir = os.getcwd() # directory ....\tesi

filename = path.join(dir,'images\\two_photons','qdot area3 12,5micron 26mW 5rep 500us') # input
savename = path.join(dir,'images\\output','qdot area3 12,5micron 26mW 5rep 500us') # output

timeSwitch = True #True if i saved the timetraces
spadFormat=7    # 7x7
Nspad = spadFormat**2 # 49
index5=[8,9,10,11,15,16,17,18,19,22,23,24,25,26,29,30,31,32,33,36,37,38,39,40] #Check if 12 is included or not (d5)
index3=[16,17,18,23,24,25,30,31,32] # d3
index1=[17,23,24,25,31] #d1

## Load Carma dataset


**SKIP ad ACCELLERATION se già eseguito una volta e si ha file in data/nomefile.npy**

per 7GB impiega circa 3 min

In [ ]:
f = h5py.File(filename + '.h5', 'r')

if timeSwitch:
    group = f['Fluctuations']
    print('Opening time data...')
    Endoftime_h5 = group['IsEndOfTimeBin'][()]
    t_bin=group['Time'][()][0]/1000        #micros
    rep = group.attrs['I_Repetitions'][0]      #repetitions  
else:
    group = f['Image']

# METADATA

imgFormat=group.attrs['I_Height'][0]       #pixels
Depth=group.attrs['I_Depth'][0]            #repetitions/z
ImgFoV=group.attrs['I_XSpan[um]'][0]       #micron
pxsizex=group.attrs['I_XPixelSize[nm]'][0]/1000 #micron         
ZSpan=group.attrs['I_ZSpan[um]'][0]        #micron
pxsizez=group.attrs['I_ZPixelSize[nm]'][0]/1000 #micron
Duration=group.attrs['M_Duration[ms]'][0]  #ms
DwellTime=Duration/(Depth*imgFormat**2)    #ms (approximate)


Fingerprint_h5 = group['Fingerprint'][()]
print('Opening SPAD data...')
data_h5= group['SPAD'][()]
print('Finished opening')
f.close()

In [ ]:
print(data_h5.shape)

### RIASSUNTO METADATA:

| **Parameter** | **Description**                             | **Unit**        |
|---------------|---------------------------------------------|-----------------|
| `imgFormat`   | Image format (e.g., 512x512)                | pixels          |
| `Depth`       | Repetitions along the Z-axis                | repetitions / z |
| `ImgFoV`      | Field of view                               | micron          |
| `pxsizex`     | Pixel size (X/Y axis)                       | micron          |
| `ZSpan`       | Z-axis span (total depth range)             | micron          |
| `pxsizez`     | Pixel size along Z-axis                     | micron          |
| `Duration`    | Time per pixel (e.g., 500 µs)               | ms              |
| `DwellTime`   | Minimum time unit (approximate)             | ms              |


### Crop timetraces overflow

In [ ]:
# tempo impiegato per 7GB circa: 1 min

# Ch,T,X,Y,R,Z
print('Full data size: ' + str(data_h5.shape[0]))
print('Total time bins (no channel distinction): ' +str(Endoftime_h5.shape[0]))

endtime=np.asarray(np.where(Endoftime_h5==1))[0,:]       #Cerco gli indici di Endoftime_h5 dove finiscono i pixels
print('Scan pixels: ' + str(endtime.shape[0]))
print('\nendtime: '+str(endtime[:10]))
v = np.max(np.diff(endtime))                             #Cerco la durata massima
print('maximum temporal timetrace length: ' +str(v))                  
overflow = endtime[np.asarray(np.where(np.diff(endtime)==v))[0,:]+1]  #Indici di cut degli elementi in overflow
print('\nOverflow: ' +str(overflow))
print('Pixels in overflow: ' + str(overflow.shape[0]))

over_del = np.zeros(((overflow.shape[0]+1)*Nspad)).astype(np.int64)
for j in range(Nspad):
    over_del[(j)*overflow.shape[0]:overflow.shape[0]*(j+1)]= ((overflow*Nspad)+j)

g=np.linspace(0,48,49).astype(int)
if endtime[0]+1==v: 
    over_del[-49:]=((endtime[0]*Nspad)+g)
else:
    over_del=np.delete(over_del, np.linspace(over_del.shape[0]-Nspad, over_del.shape[0]-1).astype(int) , axis=(0))
    
print('Bin in data_h5 to be removed: ' +str(over_del))

data_crop = np.delete(data_h5, over_del)
del over_del
del endtime
del g
del overflow
print('\nFinal size of cropped data: ' +str(data_crop.shape[0]))
print('Expected size: ' +str(Nspad*Depth*rep*imgFormat**2*(v-1)))

print('size cropped == expected size: ', str(data_crop.shape[0]==Nspad*Depth*rep*imgFormat**2*(v-1)))

### Reshape data

Reshape data with a (z,r,y,x,t,ch) format

In [ ]:
z_i, rep_i, y_i, x_i, t_i, ch_i = 0, 1, 2, 3, 4, 5 # indexes of reshaped data

In [ ]:
fingerprint_meta = np.reshape(Fingerprint_h5, (spadFormat,spadFormat))
if not timeSwitch:
    data=np.reshape(data_h5,(Depth,1,imgFormat,imgFormat,1,Nspad)) 
    print('Data format (z,rep,y,x,t,ch) ' +str(data.shape))
else:
    data=np.reshape(data_crop, (Depth,rep,imgFormat,imgFormat,v-1,Nspad))
    print('Data format (z,rep,y,x,t,ch) ' +str(data.shape))
    del data_crop
   
print('\nCumulative Dwell time (approx.): ' + str(DwellTime) +' ms')
print('FoV [um]: ' + str(ImgFoV))
if timeSwitch: print('Time binsize [us]: ' +str(t_bin))
print('Pixel size [um]: ' +str(pxsizex))
del data_h5

# Accelleration:

1. write data in a .npy file
2. import directly from .npy the data in the format (z,rep,y,x,t,ch)
3. load only the metadata from .h5
4. select only d3

In [ ]:
# DATA IS THE ARRAY OF THE SPAD DATA (z,rep,y,x,t,ch)
# np.save('data/qdot area3 12,5micron 26mW 5rep 500us',data)
# del data
# writing time circa 1 min

In [ ]:
# RELOAD FAST:
z_i, rep_i, y_i, x_i, t_i, ch_i = 0, 1, 2, 3, 4, 5 # indexes of reshaped data

fast_filename = path.join(dir,'data','qdot area3 12,5micron 26mW 5rep 500us.npy') # input

f = h5py.File(filename + '.h5', 'r')

print('Reloading metadata...')
if timeSwitch:
    group = f['Fluctuations']
    print('Opening time data...')
    Endoftime_h5 = group['IsEndOfTimeBin'][()]
    t_bin=group['Time'][()][0]/1000        #micros
    rep = group.attrs['I_Repetitions'][0]      #repetitions  
else:
    group = f['Image']

# METADATA

imgFormat=group.attrs['I_Height'][0]       #pixels
Depth=group.attrs['I_Depth'][0]            #repetitions/z
ImgFoV=group.attrs['I_XSpan[um]'][0]       #micron
pxsizex=group.attrs['I_XPixelSize[nm]'][0]/1000 #micron         
ZSpan=group.attrs['I_ZSpan[um]'][0]        #micron
pxsizez=group.attrs['I_ZPixelSize[nm]'][0]/1000 #micron
Duration=group.attrs['M_Duration[ms]'][0]  #ms
DwellTime=Duration/(Depth*imgFormat**2)    #ms (approximate)


Fingerprint_h5 = group['Fingerprint'][()]
f.close()

print('Reloading SPAD data...\n')
data = np.load(fast_filename)
print('Data shape: ',data.shape)
print('Data format: (z,rep,y,x,t,ch)')

To accelerate analysis, we work on single 5x5, 3x3, etc.... 
Due to memory limits, we work on single repetitions [0,R,:,:,:,i]
If more than 1 Z-plane are acquired, we also select it with the first index [Z,R,:,:,:,i]

In [ ]:
#Select a single repetition
d3 =[data[0,:,:,:,:,i] for i in index3]
d3 = np.transpose(np.asarray(d3),(1,2,3,4,0)).sum(axis=0)
print('d3 dataset: ' +str(d3.shape))
print('d3 dataset order: (x,y,t,ch)')

x_i3, y_i3, t_i3, ch_i3 = 0, 1, 2, 3 # indexes of d3 reshaped data


#OR sum over repetitions

#d3 = np.sum(data[0,:,:,:,:,:],axis=0)  
#d3 = [d[:,:,:,i] for i in index3]
#d3 = np.transpose(np.asarray(d3),(1,2,3,0))
#print('d3 dataset: ' +str(d3.shape))

# Check Dataset

### Check fingerprint

In [ ]:
fingerprint_meta = np.reshape(Fingerprint_h5, (spadFormat,spadFormat))

fingerprint = tool.fingerprint(data.sum(axis=(0,1,4)))
print(fingerprint.shape)
fingerprint[1,5]=(fingerprint[1,4]+fingerprint[1,6])/2 #smooth spad 12
fingerprint_meta[1,5]=(fingerprint_meta[1,4]+fingerprint_meta[1,6])/2
c= int((spadFormat-1)/2)  #change it if the maximum is not at the center

fig,ax= plt.subplots(1,2, figsize=(10,10))
ax[0].imshow(fingerprint, cmap='hot')
ax[1].imshow(fingerprint_meta, cmap='hot')
#plt.colorbar()
for i in range(7):
    for j in range(7):
        ax[0].text(j-0.3,i, str(round(fingerprint[i,j]*100/fingerprint[c,c])) + '%', color='limegreen')
        ax[0].set_title('Experimental fingerprint')
        ax[1].text(j-0.3,i, str(round(fingerprint_meta[i,j]*100/fingerprint_meta[c,c])) + '%', color='limegreen')
        ax[1].set_title('Software fingerprint')

#fig.savefig(savename + ' fingeprint.png')
#print(fingerprint)
#im = Image.fromarray(fingerprint)
#im.save(savename + ' fingerprint tif.tiff')

### Check Images stack

In [ ]:
fig_1 = gra.ShowDataset(np.sum(data,axis=(z_i,rep_i,t_i)),normalize = False, colorbar=False)

To save a stack (timetraces, all channels, etc..) Fiji wants time/channel as first index, so we use transpose. <br><br> If you want to save the timetrace of SPAD X from repetition R, it's: $\;\;$ "np.transpose(data[0,R,:,:,:,X], axes=(2,0,1))" <br>  If you want to save the NxN images, it's:  $\;\;$ "np.transpose(d3.sum(axis=2), axes=(2,0,1))" <br> Careful about the difference between Data and d3/d5/etc.!

In [ ]:
#Save stacks (x,y,t,ch) # 4 = spad centrale
tifffile.imwrite(savename + ' central timestrace.tiff', np.transpose(d3[:,:,:,4], axes=(x_i3,y_i3,t_i3))) # stack temporale canale centrale (ripetizioni sommate)

In [ ]:
#Save single images/stack (z,rep,y,x,t,ch) # 24 = canale centrale
im = Image.fromarray(data[0,0,:,:,:,24].sum(axis=rep_i).astype(np.uint8)) # stack temporale del canale centrale (sommando sulle ripetizioni)
im.save(savename + ' central.tiff')

# Image reconstruction

The available datasets are: 

data (z,rep,y,x,t,ch) <br> d3, d5,d1 (y,x,t,ch) <br>datacut (y,x,t,ch) (- spad 12)

## Confocal

In [ ]:
# d3 (y,x,t,ch))
# data (z,rep,y,x,t,ch)
datacut = np.sum(np.delete(data, 12, ch_i)[0,:,:,:,:],axis=(z_i))[...] #Remove SPAD 12
print('datacut: (tutti gli spad -12) ',datacut.shape, '(x,y,t,ch)')
print('d3: ',d3.shape, '(x,y,t,ch)')

In [ ]:
img_sum = d3.sum(axis=(t_i3,ch_i3)) #Sum over time and channel
gra.ShowImg(img_sum, pxsize_x = pxsizex)

In [ ]:
im = Image.fromarray(img_sum)
im.save(savename + ' confocal d3.tiff')

# time traces:

## full d3


In [ ]:
plt.hist(img_sum.flatten(), bins=27, range=(0,27), edgecolor='black', color='orange', alpha=0.7)
plt.title('Histogram of pixel values')
plt.xlabel('Pixel value')
plt.ylabel('Frequency')
plt.yscale('log')
plt.show()

In [ ]:
index_of_pixel_values_grater_than_20 = np.argwhere(img_sum > 20)

plt.imshow(img_sum, cmap='hot')
plt.scatter(index_of_pixel_values_grater_than_20[:,1], index_of_pixel_values_grater_than_20[:,0], color='blue', s=1)
plt.show()

In [ ]:
t = np.linspace(0,500,50)
for y,x in index_of_pixel_values_grater_than_20:
    # print(f"Pixel at ({x}, {y}) has value: {img_sum[y, x]}")
    frequency = d3.sum(axis=(ch_i3))[y,x,:]
    plt.plot(t,frequency)

plt.xlabel('Time bins (us)')
plt.show()

## solo spad centrale

In [ ]:
image_center = d3.sum(axis=(t_i3))[:,:,4] #Central spad

plt.hist(image_center.flatten(), bins=15, range=(0,15), edgecolor='black', color='purple', alpha=0.7)
plt.title('Histogram of pixel values')
plt.xlabel('Pixel value')
plt.ylabel('Frequency')
plt.yscale('log')
plt.show()

In [ ]:
index_of_pixel_values_grater_than_6 = np.argwhere(image_center > 8)

plt.imshow(image_center, cmap='hot')
plt.scatter(index_of_pixel_values_grater_than_6[:,1], index_of_pixel_values_grater_than_6[:,0], color='blue', s=1)
plt.show()

In [ ]:
t = np.linspace(0,500,50)
for y,x in index_of_pixel_values_grater_than_6:
    # print(f"Pixel at ({x}, {y}) has value: {img_sum[y, x]}")
    frequency = d3.sum(axis=(ch_i3))[y,x,:]
    plt.plot(t,frequency)

plt.xlabel('Time bins (us)')
plt.show()

## APR

Use a format (y,x,ch) for the data, remove time and repetions if present. Note: "shift_vec" is given in pixels, but apr.Reassignment later wants it in microns!

data -> ref = 24 <br> Datacut -> ref = 23 <br> d5 -> ref = 12 <br> d3 -> ref = 4 <br> d1 -> ref = 2

In [ ]:
#ISM image
usf = 10  #upsampling factor = subpixel precision
ref = 4 #reference image to compute the shift vectors

shift_vec, img_ism = apr.APR(d3.sum(axis=(t_i3)), usf, ref, filter_sigma=1, pxsize = pxsizex*1000) #pxsize in nm

img_ism_sum = img_ism.sum(axis=-1) # immagine sommando senza shift

fig, ax = plt.subplots(1, 2, figsize=(15,5))
gra.ShowImg(img_ism_sum, pxsize_x = pxsizex, fig = fig, ax = ax[0])
ax[0].set_title('ISM')
gra.PlotShiftVectors(shift_vec, fig = fig, ax = ax[1])
ax[1].set_title('Shift Vectors')
#ax[1].set_xlim(-400,500)
#ax[1].set_ylim(-300,400)

In [ ]:
im = Image.fromarray(img_ism_sum)
im.save(savename + ' ISM d3.tiff')

extent = ax[1].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
fig.savefig(savename + ' shift vectors d3.png', bbox_inches=extent.expanded(1.2, 1.2))

#Save all single images as a stack
#tifffile.imwrite(savename + ' shifted stack.tiff', np.transpose(img_ism, (2,0,1)))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,10))

gra.ShowImg(img_sum, pxsize_x = pxsizex, fig = fig, ax = ax[0])
ax[0].set_title('Confocal')

gra.ShowImg(img_ism_sum, pxsize_x = pxsizex, fig = fig, ax = ax[1])
ax[1].set_title('ISM')

## Deconvolution

### Open experimental PSF

Open a tiff stack of the 49 PSFs and resize them to the same pixel size and number of pixels of the image

In [ ]:
# upload PSF

psf_exp=tifffile.imread(path.join(dir,'images','output',"170nm 5micron 220mW 50rep stack (PSF).tiff"))
psf_fov = 5 #micron
psf_format = 512
psf_exp =np.transpose(psf_exp, (1,2,0))
figx = gra.ShowDataset(psf_exp[160:360,160:360,:], normalize=False)

In [ ]:
psf_pxsize = psf_fov/psf_format
rap=pxsizex/psf_pxsize
l= round(psf_format/rap)
psf_exp=cv2.resize(psf_exp.astype(float), (l,l))
psf_pxsize = psf_fov/psf_exp.shape[0]

pd= round((img_sum.shape[0]-psf_exp.shape[0])/2)
pd2=(img_sum.shape[0]-psf_exp.shape[0])-pd
psf_exp=np.pad(psf_exp,((pd,pd2),(pd,pd2),(0,0)), 'constant')
print("PSF format: " + str(psf_exp.shape))
print("image format: " + str(data.shape))
print("PSF pixel: " + str(psf_pxsize))
print("img pixel: " + str(pxsizex))

 <br>Define psfcut, psf_d3, etc. like for the image

In [ ]:
psfcut=np.delete(psf_exp, 12,2) #Remove SPAD 12
print(psfcut.shape)

psf_d3 =np.transpose(np.asarray([psf_exp[:,:,i] for i in index3]),axes=(1,2,0))
print(psf_d3.shape)

In [ ]:
fig_x = gra.ShowDataset(psf_d3[200:300,200:300,:],normalize = False, colorbar=False)

### Deconvolve

if out='all' the result is a stack of the iterations, select img_deconv[-1,x,y]. If out='last',it gives only the final iteration [x,y]

In [ ]:
print(psf_d3.shape)
print(d3.shape)

In [ ]:
#classical deconvolutin
img_deconv = dec.MultiImg_RL_FFT(psfcut, datacut.sum(axis=(2)), max_iter = 15, reg=0, out='all') #all or last
gra.ShowImg(img_deconv[-1,...], pxsize_x = pxsizex)

In [ ]:
print(img_sum.shape)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,20))
gra.ShowImg(img_sum[100:250,0:150], pxsize_x = pxsizex, fig = fig, ax = ax[0])
ax[0].set_title('Confocal')
gra.ShowImg(img_ism_sum[100:250,0:150], pxsize_x = pxsizex, fig = fig, ax = ax[1])
ax[1].set_title('ISM')
gra.ShowImg(img_deconv[-1,100:250,0:150], pxsize_x = pxsizex, fig = fig, ax = ax[2])
ax[2].set_title('Deconvolution')

In [ ]:
#Save all iterations
tifffile.imwrite(savename + ' deconv all-iter.tiff', img_deconv)

In [ ]:
im = Image.fromarray(img_deconv[-1,...])
im.save(savename + ' deconv d3 10rep.tiff')

## ISM-SRRF

In [ ]:
#Open ISS Tiff image
img_iss=np.asarray(Image.open("G:/Drive condivisi/BioPhotonics/Riccardo/Clara tesi/dati 2024 11 19 sfere 170nm xy/170nm 12,5micron 100mW 30rep 2 FR d3 k80 epsilon0.2.tiff"))
pxsizex=0.0488

Define magnification and radius as variables so every method uses the same values

In [ ]:
magn = 1
radius=10

For applying reassignment between SRRF steps, we take the shift that were previously found from the "classical" reconstruction. NOTE: if APR was done a 3x3, the shifts here will be a 3x3 as well! Pay attention that the images and the shift have the same format when doing reassignment!

In [ ]:
shift3=shift_vec/(pxsizex*1000)
print('3x3 shift vectors shape: ' + str(shift3.shape))

### 1) ISM->Radiality

Here, we take the ISM reconstructed image img_ism_sum (or the deconvolved image img_deconv) and calculate only the radiality map. <br><br> NOTE: srrf_ism_1 has an additional index for time at the beginning. When showing the image, use "srrf_ism_1[0,x,y]"

In [ ]:
srrf_ism_1 = eSRRF(np.ascontiguousarray(img_ism_sum[...]), magnification=magn,radius=7,doIntensityWeighting=True)[0]
print(srrf_ism_1.shape)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(15,15))
ax[0].imshow(img_deconv[-1,...],vmax=400)
ax[0].set_title('ISM')
ax[1].imshow(srrf_ism_1[0,...],vmax=30)
ax[1].set_title('SRRF over sumed ISM')

In [ ]:
im = Image.fromarray(img_deconv[-1,...])
im.save(savename + ' deconv.tiff')

### 2) ISM->SRRF

First, do pixel reassignment for every timeframe, obtaining a sequence of ISM images (ttrace_ism [x,y,t]). Then we measure radiality and apply the temporal analysis

For temporal analysis we have three possibilities: AVG makes an average of the radiality, VAR takes the variance, TAC2 computes the second order autocorrelation

In [ ]:
ttrace_ism = np.zeros(d3.shape)

for i in tqdm(range(ttrace_ism.shape[2])):  #iterate over time bins
    ttrace_ism[:,:,i,:]=apr.Reassignment(shift3,d3[:,:,i,:])
ttrace_ism=np.sum(ttrace_ism, axis=(-1))
print('ttrace_ism format [x,y,t]: ' +str(ttrace_ism.shape))

In [ ]:
#tifffile.imwrite(savename + ' ism d3 timetrace.tiff', np.transpose(ttrace_ism, axes=(2,0,1)))

In [ ]:
#Check that, summing over time, the image is the same to the previous reassigned image

fig, ax = plt.subplots(1, 2, figsize=(8,8))
ax[0].imshow(ttrace_ism.sum(axis=(-1)),vmax=30)
ax[0].set_title('separate reass')
ax[1].imshow(img_ism_sum,vmax=30)
ax[1].set_title('classical ism')

In [ ]:
#Here we do the radiality map and the temporal analysis

rad_ism_2 = np.transpose(np.zeros((ttrace_ism.shape)), axes=(2,0,1))
srrf_ism_2 = np.zeros(ttrace_ism.shape[:2])

rad_ism_2[:,:,:] = eSRRF(np.ascontiguousarray(np.transpose(ttrace_ism[:,:,:],axes=(2,0,1))), magnification=magn,radius=radius)[0]
srrf_ism_2[:,:] = calculate_eSRRF_temporal_correlations(rad_ism_2[...], 'AVG')

In [ ]:
plt.imshow(srrf_ism_2[...],vmin=0,vmax=0.2)
plt.colorbar()

In [ ]:
im = Image.fromarray(srrf_ism_2)
im.save(savename + ' SRRF_ismtrace.tiff')

### 3) SRRF->ISM

Here we apply SRRF to the time sequece of every spad, obtaining N SRRF images that we will then reassign

In [ ]:
rad_d3= np.transpose(np.zeros(d3.shape),axes=(2,0,1,3))
srrf_d3= np.zeros((d3.shape[0],d3.shape[1],d3.shape[3]))
print(rad_d3.shape)
print(srrf_d3.shape)

for i in range(d3.shape[3]):
    rad_d3[:,:,:,i] = eSRRF(np.ascontiguousarray(np.transpose(d3[:,:,:,i],axes=(2,0,1))), magnification=magn,radius=radius)[0]
    srrf_d3[:,:,i] = calculate_eSRRF_temporal_correlations(rad_d3[:,:,:,i], 'AVG')

In [ ]:
fig_1 = gra.ShowDataset(srrf_d3,normalize = False, colorbar=False)

In [ ]:
# Compare the central element intensity image with its SRRF counterpart

fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].imshow(d3.sum(axis=(2))[0:250,250:500,4], vmax=3)
ax[0].set_title('central element')
ax[1].imshow(srrf_d3[0:250,250:500,4], vmax=0.0000003)
ax[1].set_title('central SRRF')

Here we do the reassignment. NOTE: We can use shift3 only if the magnification is 1! if magnification>1, we have to do apr.APR and re-compute the shift, or rescale the shifts in some way

In [ ]:
srrf_ism_3 =apr.Reassignment(shift3,srrf_d3)
srrf_ism_3 = srrf_ism_3.sum(axis=-1)

fig, ax = plt.subplots(1, 2, figsize=(15,5))
ax[0].imshow(srrf_ism_3[...],vmax=0.02)
ax[0].set_title('Reassigned SRRF')
ax[1].imshow(img_ism_sum, vmax=50)
ax[1].set_title('ISM comparison')

In [ ]:
im = Image.fromarray(srrf_ism_3)
im.save(savename + ' SRRF-apr.tiff')


## Results

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(20,10))
plt.subplots_adjust(hspace=0.2)

ax1 = ax[0,0].imshow(img_sum[300:450,0:150])
ax[0,0].set_title('Confocal')
fig.colorbar(ax1,fraction=0.046, pad=0.04)

ax2=ax[0,1].imshow(img_ism_sum[300:450,0:150])
ax[0,1].set_title('ISM')
fig.colorbar(ax2,fraction=0.046, pad=0.04)

ax3=ax[0,2].imshow(img_sofi_sum[300:450,0:150], vmax =15000)
ax[0,2].set_title('SOFISM')
fig.colorbar(ax3,fraction=0.046, pad=0.04)

ax4=ax[1,0].imshow(srrf_ism_1[0,300:450,0:150], vmax=200)
ax[1,0].set_title('ISM -> Radiality')
fig.colorbar(ax4,fraction=0.046, pad=0.04)

ax5=ax[1,1].imshow(srrf_ism_2[300:450,0:150])
ax[1,1].set_title('APR -> SRRF')
fig.colorbar(ax5,fraction=0.046, pad=0.04)

ax6=ax[1,2].imshow(srrf_ism_3[300:450,0:150], vmax=0.8)
ax[1,2].set_title('SRRF -> APR')
fig.colorbar(ax6,fraction=0.046, pad=0.04)

# Resolution measurement

In [ ]:
img_1 = img_deconv[-1,...]
img_2 = srrf_ism_1[0,...]

In [ ]:
img_1=tifffile.imread("C:/Users/rbolz/Downloads/qdots confocal repsum.tif")
img_2=img_deconv[-1,...]

## Find peaks

In [ ]:
xc=peak_local_max(img_1,min_distance=5, threshold_abs=15)

maxvalues=np.zeros(xc.shape[0],dtype='float')
for i in range(xc.shape[0]):
    maxvalues[i]=img_1[xc[i,0],xc[i,1]]

    xc1=np.zeros((xc.shape[0],2))
n=0
for i in range(xc.shape[0]):
    if 200>maxvalues[i]>80: #Change value range
        xc1[n,:]=xc[i,:]
        n+=1       
xc1=xc1[0:n,:]

In [ ]:
ROIsizeC = 30
Npart = xc1.shape[0]
fig, ax = plt.subplots(1, 2, figsize=(15,15))

ax[0].imshow(img_1,cmap='hot', vmax=300)
#ax[0].scatter(xc1[:,1],xc1[:,0])
ax[0].set_title('Confocal')
for l in range(Npart):
    rect = Rectangle((xc1[l,1]-ROIsizeC/2,xc1[l,0]-ROIsizeC/2),ROIsizeC,ROIsizeC,linewidth=1,edgecolor='w',facecolor='none')
    ax[0].add_patch(rect)
    ax[0].text(xc1[l,1]+4+ROIsizeC/2, xc1[l,0], l, fontsize=14, color='green')
    
ax[1].imshow(img_2,cmap='hot', vmax=50)
ax[1].set_title('ISM')
for l in range(Npart):
    rect = Rectangle((xc1[l,1]-ROIsizeC/2,xc1[l,0]-ROIsizeC/2),ROIsizeC,ROIsizeC,linewidth=1,edgecolor='w',facecolor='none')
    ax[1].add_patch(rect)
    ax[1].text(xc1[l,1]+4+ROIsizeC/2, xc1[l,0], l, fontsize=14, color='white')   
ax = plt.gca()

In [ ]:
# Remove specific particles
#Refresh the image above to see the updated indexes after every deletion!

xc1=np.delete(xc1,[2,12],0)
Npart = xc1.shape[0]

## Particle Fit

In [ ]:
def erfC(x, amp, xc, w, offset):
    fitfunction=amp*w*w*np.pi*(1-scipy.special.erf((xc-x)/(np.sqrt(2.0)*w)))+offset*ROIsizeC*x
    return fitfunction

In [ ]:
res_conf=np.zeros((Npart,3))
res_ism=np.zeros((Npart,3))
integrali_conf=np.zeros((Npart,ROIsizeC,2))
integrali_ism=np.zeros((Npart,ROIsizeC,2))

for l in range(0,Npart,1): #inizia il loop sulle particelle
    
    ROItemp_conf=np.zeros((ROIsizeC,ROIsizeC),dtype='float')
    ROItemp_ism=np.zeros((ROIsizeC,ROIsizeC),dtype='float')
    yi= (xc1[l,1]-ROIsizeC/2).astype(int)
    yf= (xc1[l,1]+ROIsizeC/2).astype(int)
    xi= (xc1[l,0]-ROIsizeC/2).astype(int)
    xf= (xc1[l,0]+ROIsizeC/2).astype(int)
    
    for i in range(xi,xf,1): #crea la ROI attorno alla NP
        for j in range(yi,yf,1):
            ROItemp_conf[i-xi][j-yi]=img_1[i][j]
            ROItemp_ism[i-xi][j-yi]=img_2[i][j]
    #plt.imshow(ROItemp_conf.T)
    
    integral_conf=np.zeros(ROIsizeC+1,dtype='float')
    integral_ism=np.zeros(ROIsizeC+1,dtype='float')
    axisC=np.zeros(ROIsizeC+1,dtype='float')
    
    for i in range(ROIsizeC+1):
        axisC[i]=i
        
    for i in range(ROIsizeC):   
        for j in range(ROIsizeC):
            integral_conf[i]=integral_conf[i]+ROItemp_conf[j][i]
            integral_ism[i]=integral_ism[i]+ROItemp_ism[j][i]
        integral_conf[i+1]=integral_conf[i]
        integral_ism[i+1]=integral_ism[i]
            
    popt1, pcov1 = curve_fit(erfC, axisC, integral_conf, bounds=(0,np.inf), p0=(5,25,3,0))
    popt2, pcov2 = curve_fit(erfC, axisC, integral_ism, bounds=(0,np.inf), p0=(5,15,3,0))
    
    res_conf[l,0]=res_ism[l,0]=l
    res_conf[l,1]=popt1[2]*pxsizex*2.35/magn
    res_ism[l,1]=popt2[2]*pxsizex*2.35/magn
    res_conf[l,2]=popt1[0]
    res_ism[l,2]=popt2[0]
    
    
    for i in range(ROIsizeC):
        integrali_conf[l,i,0]=integral_conf[i]
        integrali_conf[l,i,1]=erfC(axisC,*popt1)[i]
        integrali_ism[l,i,0]=integral_ism[i]
        integrali_ism[l,i,1]=erfC(axisC,*popt2)[i]
    
print('Confocal Avg FWHM: \n' + str(np.mean(res_conf[:,1])) + ' +- ' + str(np.std(res_conf[:,1])) + '\n\n ISM Avg FWHM: \n' + str(np.mean(res_ism[:,1])) + ' +- ' + str(np.std(res_ism[:,1])))
print('\nConfocal Intensity: \n' + str(np.mean(res_conf[:,2])) + ' +- ' + str(np.std(res_conf[:,2])) + '\n\n ISM Intensity: \n' + str(np.mean(res_ism[:,2])) + ' +- ' + str(np.std(res_ism[:,2])))

In [ ]:
print( 'Confocal results: \n' + str(res_conf) + '\n\n ISM results: \n' + str(res_ism))

In [ ]:
plt.plot(integrali_conf[1,:,0],marker='o')
plt.plot(integrali_conf[1,:,1],linestyle='dashed')
plt.tick_params(axis='both', direction= 'in')

## pROVE CORRELAZIONI

In [ ]:
print(d3.shape)

In [ ]:
#Definisco sequenza intensità ISM

ttrace_ism = np.zeros((3,512,512,50,9))

for i in tqdm(range(50)):
    ttrace_ism[0,:,:,i,:]=apr.Reassignment(shift3,d3[0,:,:,i,:])
    ttrace_ism[1,:,:,i,:]=apr.Reassignment(shift3,d3[1,:,:,i,:])
    ttrace_ism[2,:,:,i,:]=apr.Reassignment(shift3,d3[2,:,:,i,:])
ttrace_ism_sum=np.sum(ttrace_ism, axis=(-1))
print(ttrace_ism_sum.shape)

In [ ]:
plt.plot(ttrace_ism_sum[0,440,332,:])

In [ ]:
#SRRF sulla sequenza di intensità
srrf_ism_2_var = np.zeros((3,512,512))
srrf_ism_2_avg = np.zeros((3,512,512))
srrf_ism_2_tac = np.zeros((3,512,512))

for f in range(3):
    srrf_ism_sum_2= eSRRF(np.ascontiguousarray(np.transpose(ttrace_ism_sum[f,:,:,:],axes=(2,0,1))), magnification=1,radius=7)[0]
    srrf_ism_2_var[f,...] = calculate_eSRRF_temporal_correlations(srrf_ism_sum_2, 'VAR')
    srrf_ism_2_avg[f,...] = calculate_eSRRF_temporal_correlations(srrf_ism_sum_2, 'AVG')
    srrf_ism_2_tac[f,...] = calculate_eSRRF_temporal_correlations(srrf_ism_sum_2, 'TAC2')

In [ ]:
print(srrf_ism_2_tac.shape)

In [ ]:
srrf_ism_2_var_mean = np.mean(srrf_ism_2_var, axis=0)
srrf_ism_2_avg_mean = np.mean(srrf_ism_2_avg, axis=0)
srrf_ism_2_tac_mean = np.abs(np.mean(srrf_ism_2_tac, axis=0))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,10))
ax0 = ax[0].imshow(srrf_ism_2_avg_mean[430:455, 320:345])
ax[0].set_title('average')
ax1= ax[1].imshow(srrf_ism_2_var_mean[430:455, 320:345])
ax[1].set_title('Variance')
ax2 = ax[2].imshow(srrf_ism_2_tac_mean[430:455, 320:345])
ax[2].set_title('Autocorr.')
print(srrf_ism_sum_2.shape)

In [ ]:
def correla(t1,tau):
    corr_carota=0
    t1_b = t1 - np.mean(t1)
    
    for i in range(t1.shape[0]-tau):
        corr_carota += t1_b[i]*t1_b[i+tau]
    corr_carota /= t1.shape[0]-tau
    corr_carota /= np.mean(t1)**2
    return corr_carota

In [ ]:
img_autocorr=np.zeros((512,512,2))
for i in tqdm(range(srrf_ism_sum_2.shape[1])):
    for j in range(srrf_ism_sum_2.shape[2]):
        for tau in range(2):
            img_autocorr[i,j,tau] = correla(srrf_ism_sum_2[:,i,j],tau)

In [ ]:
plt.imshow(img_autocorr[:,:,1])

In [ ]:
#Correlazioni definite da noi

srrf_carota = srrf_ism_sum_2[:,441,334]
srrf_carota_delta = srrf_carota - np.mean(srrf_carota)
corr_carota=0

for i in range(srrf_carota.shape[0]-1):
    corr_carota += srrf_carota_delta[i]*srrf_carota_delta[i+1]
corr_carota /= srrf_carota.shape[0]-1
#corr_carota /= np.mean(srrf_carota)**2

print(corr_carota)
print(srrf_ism_2[441,334])

In [ ]:
#Correlazioni con Scipy

test2 = np.zeros((ttrace_ism_sum.shape[0],ttrace_ism_sum.shape[1],ttrace_ism_sum.shape[2]))

for i in range(d3.shape[1]):
    for j in range(d3.shape[2]):
        m4 = np.mean(ttrace_ism_sum[i,j,:])
        test2[i,j,:] = scipy.signal.correlate(ttrace_ism_sum[i,j,:]-m4,ttrace_ism_sum[i,j,:]-m4, method='auto', mode='same')
        if m4==0: m4=1
        test2[i,j,:] /= m4**2
plt.imshow(test2[430:455, 320:345,:].sum(axis=2))
plt.colorbar()

In [ ]:
plt.plot(test2[440,:,:].sum(axis=1))